## Maximum theoretical yield of yanuthone D production

### Import the required libraries and the model.

In [90]:
from cobra.io import read_sbml_model
from cobra import Model, Reaction, Metabolite
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline


model = read_sbml_model('2018_ATCC1015_GSM_iJB1325.xml') #The model is refered from (Brandl, J., Aguilar-Pontes, M. V., Schape, P., Norregaard, A., Arvas, M., Ram, A. F. J., Meyer, V., Tsang, A., de Vries, R. P., & Andersen, M. R. (2018). A community-driven reconstruction of the Aspergillus niger metabolic network. Fungal Biology and Biotechnology, 5, 16. )

C:\Users\boysa\AppData\Local\Temp\ipykernel_26548\189181901.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
'' is not a valid SBML 'SId'.


### Maximum theoretical yield using default media


#### Default meduim in the model

In [54]:
model.medium #look into the default medium in the model

{'BOUNDARY_GLCe': 10.0,
 'BOUNDARY_H2Oe': 1000.0,
 'BOUNDARY_HNO3e': 1000.0,
 'BOUNDARY_O2e': 1000.0,
 'BOUNDARY_PIe': 1000.0,
 'BOUNDARY_SLFe': 1000.0,
 'BOUNDARY_Ke': 1000.0,
 'BOUNDARY_CAe': 1000.0}

#### Calculation of theoritical yield in (mmol/mmol):

In [95]:
model.objective = model.reactions.r1976 #setting model objective
yanD_production = model.optimize().objective_value
print('Maximum productivity =', yanD_production, 'mmol/gDW*h')
   
maximum_yield = yanD_production / (-1*(model.reactions.BOUNDARY_GLCe.flux))
print('Maximum theoretical yield =', maximum_yield, 'mmol-yanD/mmol-glc')

Maximum productivity = 22.910417782538882 mmol/gDW*h
Maximum theoretical yield = 1.5273611855025921 mmol-yanD/mmol-glc


#### Calculation of theoritical yield in (cmol/cmol): 

In [96]:
nC_vanD = 28 #number of C atom in yanuthone D
nC_glc = 6 #number of C atom in glucose
yanD_production_cmol = (maximum_yield*nC_vanD)/(1*nC_glc)
print('Maximum theoretical yield =', yanD_production_cmol, 'cmol-yanD/cmol-glc')

Maximum theoretical yield = 7.1276855323454305 cmol-yanD/cmol-glc


### Maximum theoretical yield using YES media

#### Introduce boundary conditiones

In [115]:
#Add extracellular metanolites
PNTOe = Metabolite('PNTOe', formula='C9H17NO5', name='(R)-Pantothenate', compartment='e') 
model.add_metabolites([PNTOe])

DHFe = Metabolite('DHFe', formula='C19H21N7O6', name='7,8-Dihydrofolate', compartment='e') 
model.add_metabolites([DHFe])

PABAe = Metabolite('PABAe', formula='C7H7NO2', name='4-Aminobenzoate', compartment='e') 
model.add_metabolites([PABAe])

PLe = Metabolite('PLe', formula='C8H9NO3', name='Pyridoxal', compartment='e') 
model.add_metabolites([PLe])

RIBOFLAVe = Metabolite('RIBOFLAVe', formula='C17H20N4O6', name='Riboflavin', compartment='e') 
model.add_metabolites([RIBOFLAVe])

#Add transport reactions
r2392 = Reaction('r2392')
r2392.name = '(R)-Pantothenate transport'
r2392.subsystem = "Transport"
r2392.lower_bound = -1000
r2392.add_metabolites({PNTOe: -1.0, model.metabolites.PNTO: 1.0})

r2393 = Reaction('r2393')
r2393.name = '7,8-Dihydrofolate transport'
r2393.lower_bound = -1000
r2393.subsystem = 'Transport'
r2393.add_metabolites({DHFe: -1.0, model.metabolites.DHF: 1.0})

r2394 = Reaction('r2394')
r2394.name = '4-Aminobenzoate transport'
r2394.lower_bound = -1000
r2394.subsystem = 'Transport'
r2394.add_metabolites({PABAe: -1.0, model.metabolites.PABA: 1.0})

r2395 = Reaction('r2395')
r2395.name = 'Pyridoxal transport'
r2395.lower_bound = -1000
r2395.subsystem = 'Transport'
r2395.add_metabolites({PLe: -1.0, model.metabolites.PL: 1.0})

r2396 = Reaction('r2396')
r2396.name = 'Riboflavin transport'
r2396.lower_bound = -1000
r2396.subsystem = 'Transport'
r2396.add_metabolites({RIBOFLAVe: -1.0, model.metabolites.RIBOFLAV: 1.0})

r2397 = Reaction('r2397')
r2397.name = 'Copper transport'
r2397.lower_bound = -1000
r2397.subsystem = 'Transport'
r2397.add_metabolites({model.metabolites.CUe: -1.0, model.metabolites.CU: 1.0})

#Add boundary conditions
model.add_boundary(model.metabolites.PNTOe, type="exchange", reaction_id= "BOUNDARY_PNTOe")
model.add_boundary(model.metabolites.DHFe, type="exchange", reaction_id= "BOUNDARY_DHFe")
model.add_boundary(model.metabolites.PABAe, type="exchange", reaction_id= "BOUNDARY_PABAe")
model.add_boundary(model.metabolites.PLe, type="exchange", reaction_id= "BOUNDARY_PLe")
model.add_boundary(model.metabolites.RIBOFLAVe, type="exchange", reaction_id= "BOUNDARY_RIBOFLAVe")
model.add_boundary(model.metabolites.CUe, type="exchange", reaction_id= "BOUNDARY_CUe")

Reaction identifier,BOUNDARY_CUe
Name,Copper exchange
Memory address,0x2349a006c50
Stoichiometry,CUe <=> Copper <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


#### Formulating the YES media: 
The undefined composition of yeast extract in Yeast-Extract with Supplments (YES) media was assumed to be that of YNB media plus 20 amino acids and d-glucose (Suthers et al., 2020). 

In [135]:
medium_blank = {key: 0 for key in model.medium}
YESmedium = medium_blank # Setup the object for YESmedium

#Set the concentration for the amino acids; assumed to be half of the initial concentration of YESmedium
YESmedium['BOUNDARY_GLCe'] = 15 
YESmedium['BOUNDARY_ALAe'] = 10
YESmedium['BOUNDARY_ARGe'] = 10
YESmedium['BOUNDARY_ASPe'] = 10
YESmedium['BOUNDARY_CYSe'] = 10
YESmedium['BOUNDARY_GLUe'] = 10
YESmedium['BOUNDARY_GLYe'] = 10
YESmedium['BOUNDARY_HISe'] = 10
YESmedium['BOUNDARY_ILEe'] = 10
YESmedium['BOUNDARY_LEUe'] = 10
YESmedium['BOUNDARY_LYSe'] = 10
YESmedium['BOUNDARY_METe'] = 10
YESmedium['BOUNDARY_PHEe'] = 10
YESmedium['BOUNDARY_PROe'] = 10
YESmedium['BOUNDARY_SERe'] = 10
YESmedium['BOUNDARY_THRe'] = 10
YESmedium['BOUNDARY_TRPe'] = 10
YESmedium['BOUNDARY_TYRe'] = 10
YESmedium['BOUNDARY_VALe'] = 10

#Set the concentration for the supplements; assumed to be half of the initial concentration of YESmedium (https://www.usbio.net/media/Y2025)
YESmedium['BOUNDARY_NH4e'] = 2.5 #Ammonium  
YESmedium['BOUNDARY_SLFe'] = 2.75 #Sulfate
YESmedium['BOUNDARY_BTe'] = 1/10**6 #Biotin 
YESmedium['BOUNDARY_PNTOe'] = 200/10**6 #Pantothenic Acid
YESmedium['BOUNDARY_DHFe'] = 1/10**6 #Folic acid
YESmedium['BOUNDARY_MYOIe'] = 2000/10**6 #myo-Inositol
YESmedium['BOUNDARY_NICAe'] = 400/10**6 #Nicotinate
YESmedium['BOUNDARY_PABAe'] = 200/10**6 #4-Aminobenzoate
YESmedium['BOUNDARY_PLe'] = 400/10**6 #Pyridoxal
YESmedium['BOUNDARY_RIBOFLAVe'] = 200/10**6 #Riboflavin
YESmedium['BOUNDARY_THIAe'] = 400/10**6 #Thiamine
YESmedium['BOUNDARY_CUe'] = 40/10**6 #Copper
YESmedium['BOUNDARY_Ke'] = 0.5 #Potassium
YESmedium['BOUNDARY_FE2e'] = 100/10**6 #Ferric 
YESmedium['BOUNDARY_MNe'] = 200/10**6 #Manganese
YESmedium['BOUNDARY_MOe'] = 100/10**6 #Molybdate
YESmedium['BOUNDARY_ZNe'] = 200/10**6 #Zinc
YESmedium['BOUNDARY_PIe'] = 0.5 #Phosphate
YESmedium['BOUNDARY_CAe'] = 0.2 #Calcium 

#Set the condition
YESmedium['BOUNDARY_H2Oe'] = 1000 #Water
YESmedium['BOUNDARY_O2e'] = 1000 #Oxygen 


#### Setting the bounds of the exchange reactions in order to create the fluxes:

In [136]:
model.reactions.BOUNDARY_GLCe.bounds = -15, 1000
model.reactions.BOUNDARY_ALAe.bounds = -10, 0
model.reactions.BOUNDARY_ARGe.bounds = -10, 0
model.reactions.BOUNDARY_ASPe.bounds = -10, 0
model.reactions.BOUNDARY_CYSe.bounds = -10, 0
model.reactions.BOUNDARY_GLUe.bounds = -10, 0
model.reactions.BOUNDARY_GLYe.bounds = -10, 0
model.reactions.BOUNDARY_HISe.bounds = -10, 0
model.reactions.BOUNDARY_ILEe.bounds = -10, 0
model.reactions.BOUNDARY_LEUe.bounds = -10, 0
model.reactions.BOUNDARY_LYSe.bounds = -10, 0
model.reactions.BOUNDARY_METe.bounds = -10, 0
model.reactions.BOUNDARY_PHEe.bounds = -10, 0
model.reactions.BOUNDARY_PROe.bounds = -10, 0
model.reactions.BOUNDARY_SERe.bounds = -10, 0
model.reactions.BOUNDARY_THRe.bounds = -10, 0
model.reactions.BOUNDARY_TRPe.bounds = -10, 0
model.reactions.BOUNDARY_TYRe.bounds = -10, 0
model.reactions.BOUNDARY_VALe.bounds = -10, 0

In [137]:
YESmedium

{'BOUNDARY_ALAe': 10,
 'BOUNDARY_ARGe': 10,
 'BOUNDARY_ASPe': 10,
 'BOUNDARY_BTe': 1e-06,
 'BOUNDARY_CYSe': 10,
 'BOUNDARY_GLCe': 15,
 'BOUNDARY_GLUe': 10,
 'BOUNDARY_GLYe': 10,
 'BOUNDARY_H2Oe': 1000,
 'BOUNDARY_HISe': 10,
 'BOUNDARY_ILEe': 10,
 'BOUNDARY_LEUe': 10,
 'BOUNDARY_LYSe': 10,
 'BOUNDARY_METe': 0,
 'BOUNDARY_NICAe': 0.0004,
 'BOUNDARY_O2e': 1000,
 'BOUNDARY_PHEe': 10,
 'BOUNDARY_PIe': 0.5,
 'BOUNDARY_SERe': 10,
 'BOUNDARY_SLFe': 2.75,
 'BOUNDARY_THRe': 10,
 'BOUNDARY_TRPe': 10,
 'BOUNDARY_TYRe': 10,
 'BOUNDARY_VALe': 10,
 'BOUNDARY_Ke': 0.5,
 'BOUNDARY_CAe': 0.2,
 'BOUNDARY_NH4e': 2.5,
 'BOUNDARY_MYOIe': 0.002,
 'BOUNDARY_MNe': 0.0002,
 'BOUNDARY_THIAe': 0.0004,
 'BOUNDARY_MOe': 0.0001,
 'BOUNDARY_ZNe': 0.0002,
 'BOUNDARY_FE2e': 0.0001,
 'BOUNDARY_PNTOe': 0.0002,
 'BOUNDARY_DHFe': 1e-06,
 'BOUNDARY_PABAe': 0.0002,
 'BOUNDARY_PLe': 0.0004,
 'BOUNDARY_RIBOFLAVe': 0.0002,
 'BOUNDARY_CUe': 4e-05}

#### Computing the summed carbon source fluxes: 

In [138]:
CSource_fluxes = [model.reactions.BOUNDARY_ALAe.flux,
       model.reactions.BOUNDARY_ARGe.flux,
       model.reactions.BOUNDARY_ASPe.flux,
       model.reactions.BOUNDARY_CYSe.flux,
       model.reactions.BOUNDARY_GLUe.flux,
       model.reactions.BOUNDARY_GLYe.flux,
       model.reactions.BOUNDARY_HISe.flux,
       model.reactions.BOUNDARY_ILEe.flux,
       model.reactions.BOUNDARY_LEUe.flux,
       model.reactions.BOUNDARY_LYSe.flux,
       model.reactions.BOUNDARY_METe.flux,
       model.reactions.BOUNDARY_PHEe.flux,
       model.reactions.BOUNDARY_PROe.flux,
       model.reactions.BOUNDARY_SERe.flux,
       model.reactions.BOUNDARY_THRe.flux,
       model.reactions.BOUNDARY_TRPe.flux,
       model.reactions.BOUNDARY_TYRe.flux,
       model.reactions.BOUNDARY_VALe.flux,
       model.reactions.BOUNDARY_GLCe.flux ]

In [139]:
sum_CSource_fluxes = sum(CSource_fluxes)

#### Computing the maximum theoretical yield using YES media (mmol/mmol):

In [140]:
model.medium = YESmedium
with model:
    model.objective = model.reactions.r1976
    yanD_production = model.optimize().objective_value
    print('Maximum productivity =', yanD_production, 'mmol/gDW*h')
   
    maximum_yield_YES = yanD_production / (-1*(sum_CSource_fluxes))
    print('Maximum theoretical yield =', maximum_yield_YES, 'mmol-yanD/mmol-Csources')

Maximum productivity = 20.515486770219624 mmol/gDW*h
Maximum theoretical yield = 0.12433628345587651 mmol-yanD/mmol-Csources
